In [29]:
import yfinance as yf
import pandas as pd
import time
import pyodbc
import requests
from tqdm import tqdm

# Step 1: Connect to SQL Server
server = 'localhost\\SQLEXPRESS'  # Server name (as configured during installation)
database = 'FinanceDB'            # SQL Server database name
connection_string = f"""
DRIVER={{ODBC Driver 17 for SQL Server}};
SERVER={server};
DATABASE={database};
Trusted_Connection=yes;
"""

# Try to establish a connection
try:
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    print("✅ Successfully connected to the database")
except Exception as e:
    print("❌ Connection error:", e)

# Step 2: Create the StockPrices table if it doesn't exist
create_table_query = """
IF NOT EXISTS (
    SELECT * FROM sysobjects WHERE name='StockPrices' AND xtype='U'
)
CREATE TABLE StockPrices (
    [Date] DATE,
    [Open] FLOAT,
    [High] FLOAT,
    [Low] FLOAT,
    [Close] FLOAT,
    [Volume] BIGINT,
    [Ticker] VARCHAR(10)
);
"""

# Execute the query
cursor.execute(create_table_query)
conn.commit()
print("✅ Table 'StockPrices' is ready")

# Step 3: Define the insert SQL query
insert_query = """
INSERT INTO StockPrices ([Date], [Open], [High], [Low], [Close], [Volume], [Ticker])
VALUES (?, ?, ?, ?, ?, ?, ?)
"""

# Step 4: Function to get the list of S&P 500 tickers from Wikipedia
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    tickers = tables[0]["Symbol"].tolist()
    tickers = [ticker.replace(".", "-") for ticker in tickers]  # Yahoo Finance uses '-' instead of '.'
    return tickers

# Step 5: Download and insert each stock's data one-by-one
def download_and_insert_to_sql(start_date="1900-01-01", pause=1.5):
    tickers = get_sp500_tickers()

    for ticker in tqdm(tickers, desc="📥 Downloading and inserting"):
        try:
            df = yf.download(ticker, start=start_date, progress=False)
            if not df.empty:
                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = df.columns.get_level_values(0)
                df.reset_index(inplace=True)
                df["Ticker"] = ticker
                for _, row in df.iterrows():
                    cursor.execute(insert_query, (
                        row["Date"], row["Open"], row["High"], row["Low"],
                        row["Close"], row["Volume"], row["Ticker"]
                    ))
                conn.commit()
        except Exception as e:
            print(f"⚠️ Error for {ticker}: {e}")
        time.sleep(pause)  # Pause to avoid hitting API rate limits

    print("✅ All data has been inserted into SQL Server")

# Step 6: Run the download and insert process
download_and_insert_to_sql(pause=0.5)

# Step 7: Close the connection
conn.close()


✅ Successfully connected to the database
✅ Table 'StockPrices' is ready


📥 Downloading and inserting: 100%|██████████| 503/503 [22:48<00:00,  2.72s/it]

✅ All data has been inserted into SQL Server


In [35]:
def load_stock_data(ticker: str, start_date: str) -> pd.DataFrame:
    """
    Load stock data from SQL Server for a given ticker and start date.
    
    Parameters:
        ticker (str): The stock ticker symbol (e.g., 'GOOG').
        start_date (str): The start date in 'YYYY-MM-DD' format.

    Returns:
        pd.DataFrame: DataFrame with selected stock data.
    """
    # SQL Server connection settings
    server = 'localhost\\SQLEXPRESS'
    database = 'FinanceDB'
    connection_string = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    Trusted_Connection=yes;
    """

    try:
        # Connect to SQL Server
        conn = pyodbc.connect(connection_string)

        # Construct SQL query
        query = f"""
        SELECT *
        FROM StockPrices
        WHERE [Ticker] = ? AND [Date] >= ?
        ORDER BY [Date];
        """

        # Use parameters to avoid SQL injection
        df = pd.read_sql(query, conn, params=[ticker, start_date])

        print(f"✅ Data for {ticker} since {start_date} successfully loaded:")
        return df

    except Exception as e:
        print("❌ Error loading data:", e)
        return pd.DataFrame()

    finally:
        try:
            conn.close()
        except:
            pass


In [36]:
df_goog = load_stock_data("GOOG", "2000-01-01")
df_goog.tail(10)

C:\Users\m-a-a\AppData\Local\Temp\ipykernel_7960\3961759584.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=[ticker, start_date])


✅ Data for GOOG since 2000-01-01 successfully loaded:


,Date,Open,High,Low,Close,Volume,Ticker
5220,2025-05-19,165.714996,167.949997,165.414993,167.869995,21374700,GOOG
5221,2025-05-20,167.759995,169.679993,164.259995,165.320007,33563300,GOOG
5222,2025-05-21,164.949997,174.529999,164.889999,170.059998,45567600,GOOG
5223,2025-05-22,173.350006,178.130005,171.880005,171.979996,45024100,GOOG
5224,2025-05-23,170.279999,171.205002,169.259995,169.589996,24963600,GOOG
5225,2025-05-27,171.300003,174.289993,171.210007,173.979996,24341300,GOOG
5226,2025-05-28,173.979996,176.479996,173.014999,173.380005,25999200,GOOG
5227,2025-05-29,175.000000,175.399994,171.779999,172.960007,21233600,GOOG
5228,2025-05-30,172.410004,173.440002,168.524994,172.850006,36233600,GOOG
5229,2025-06-02,169.065002,171.062393,168.649994,170.369995,24654673,GOOG


In [ ]:
# SQL query to drop the StockPrices table if it exists
drop_table_query = "IF OBJECT_ID('StockPrices', 'U') IS NOT NULL DROP TABLE StockPrices;"

# Execute the drop table query
cursor.execute(drop_table_query)
conn.commit()

print("🗑️ Table 'StockPrices' has been dropped (if it existed)")